# 生鮮食品需要予測・分析システム 使用例

このノートブックでは、生鮮食品需要予測・分析システムの基本的な使用方法を説明します。

## 📋 目次
1. [環境セットアップ](#環境セットアップ)
2. [基本的な使用方法](#基本的な使用方法)
3. [個別コンポーネントの使用](#個別コンポーネントの使用)
4. [カスタマイズ例](#カスタマイズ例)
5. [結果の分析と解釈](#結果の分析と解釈)
6. [トラブルシューティング](#トラブルシューティング)

## 1. 環境セットアップ

In [ ]:
# 必要なライブラリのインポート
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# プロジェクトルートをパスに追加
project_root = Path().resolve().parent
sys.path.append(str(project_root))

# システムコンポーネントのインポート
from src.main import DemandForecastingPipeline
from src.demand_forecasting.core.data_processor import DataProcessor
from src.demand_forecasting.core.feature_engineer import FeatureEngineer
from src.demand_forecasting.core.model_builder import ModelBuilder
from src.demand_forecasting.utils.config import Config
from src.demand_forecasting.utils.logger import Logger

print("✅ 環境セットアップ完了")

## 2. 基本的な使用方法

### 2.1 全体パイプラインの実行

In [ ]:
# 設定ファイルのパス
config_path = "../config/config.yaml"

# データファイルが存在するかチェック
data_path = "../data/raw/納品用_20240101_20241231_生鮮全品data.csv"
if not Path(data_path).exists():
    print("⚠️ データファイルが見つかりません。サンプルデータを作成します...")
    # サンプルデータの作成（実際の使用時は不要）
    sample_data = pd.DataFrame({
        '商品コード': ['001', '002', '003'] * 100,
        '商品名称': ['りんご', 'キャベツ', '牛肉'] * 100,
        '年月日': pd.date_range('2024-01-01', periods=300).strftime('%Y-%m-%d'),
        '金額': np.random.randint(100, 1000, 300),
        '数量': np.random.randint(1, 10, 300),
        '平均価格': np.random.randint(50, 200, 300)
    })
    os.makedirs(Path(data_path).parent, exist_ok=True)
    sample_data.to_csv(data_path, index=False, encoding='shift_jis')
    print("✅ サンプルデータを作成しました")

In [ ]:
# パイプラインの初期化
pipeline = DemandForecastingPipeline(config_path)

# 全体分析の実行（商品数を制限してデモ実行）
print("🚀 需要予測分析を開始します...")
results = pipeline.run_full_analysis(max_products=5)

# 結果サマリーの表示
print("\n📊 分析結果サマリー")
print(f"分析商品数: {results['summary']['total_products_analyzed']}")
print(f"成功率: {results['summary']['success_rate']*100:.1f}%")
print(f"平均R²スコア: {results['summary']['average_r2']:.3f}")
print(f"生成レポート数: {len(results['report_files'])}")
print(f"生成可視化数: {len(results['visualization_files'])}")

### 2.2 結果の詳細確認

In [ ]:
# 分析結果の詳細表示
if results['analysis_results']:
    print("\n🎯 商品別分析結果")
    for i, result in enumerate(results['analysis_results'][:3], 1):  # 上位3商品のみ表示
        print(f"\n{i}. {result['product_name']}")
        print(f"   品質レベル: {result['quality_level']}")
        print(f"   実用化準備: {result['implementation_readiness']}")
        print(f"   R²スコア: {result['test_metrics']['r2_score']:.4f}")
        print(f"   RMSE: {result['test_metrics']['rmse']:.2f}")
        
        # 需要曲線分析結果（存在する場合）
        if 'demand_results' in result:
            demand = result['demand_results']
            current_price = demand.get('current_price', 0)
            optimal_price = demand.get('optimal_price', 0)
            if current_price > 0 and optimal_price > 0:
                price_change = ((optimal_price - current_price) / current_price) * 100
                print(f"   価格最適化: {current_price:.0f}円 → {optimal_price:.0f}円 ({price_change:+.1f}%)")
else:
    print("⚠️ 分析可能な商品がありませんでした")

## 3. 個別コンポーネントの使用

### 3.1 データ処理

In [ ]:
# 個別コンポーネントの使用例
config = Config(config_path)
data_processor = DataProcessor(config)

# 生データの読み込み
print("📊 データ読み込み中...")
raw_data = data_processor.load_raw_data()
print(f"読み込みデータ: {len(raw_data)}行 x {len(raw_data.columns)}列")

# データの基本情報表示
print("\n📈 データ概要:")
print(raw_data.info())

# 商品別レコード数
print("\n🏷️ 商品別レコード数（上位10商品）:")
product_counts = raw_data['商品名称'].value_counts().head(10)
print(product_counts)

In [ ]:
# データクリーニング
print("🧹 データクリーニング実行中...")
clean_data = data_processor.clean_data(raw_data)
print(f"クリーニング後: {len(clean_data)}行")

# クリーニング結果の比較
print(f"削除されたレコード数: {len(raw_data) - len(clean_data)}")
print(f"欠損値: {clean_data.isnull().sum().sum()}個")

### 3.2 特徴量エンジニアリング

In [ ]:
# 特徴量エンジニアリング
feature_engineer = FeatureEngineer(config)

print("⚙️ 特徴量生成中...")

# ベースライン特徴量
baseline_features = feature_engineer.create_baseline_features(clean_data)
print(f"ベースライン特徴量追加後: {len(baseline_features.columns)}列")

# 時間特徴量
time_features = feature_engineer.add_time_features(baseline_features)
print(f"時間特徴量追加後: {len(time_features.columns)}列")

# 気象特徴量（注意：API呼び出しまたはフォールバック）
try:
    final_features = feature_engineer.integrate_weather_features(time_features)
    print(f"気象特徴量追加後: {len(final_features.columns)}列")
except Exception as e:
    print(f"⚠️ 気象特徴量追加でエラー: {e}")
    final_features = time_features

# 追加された特徴量の確認
new_columns = set(final_features.columns) - set(raw_data.columns)
print(f"\n🆕 追加された特徴量 ({len(new_columns)}個):")
for col in sorted(new_columns):
    print(f"  - {col}")

### 3.3 モデル構築と評価

In [ ]:
# 特定商品のモデル構築例
model_builder = ModelBuilder(config)

# 分析対象商品を選択（データ量の多い商品）
product_counts = final_features['商品名称'].value_counts()
target_product = product_counts.index[0]  # 最も多い商品
product_data = final_features[final_features['商品名称'] == target_product].copy()

print(f"🎯 対象商品: {target_product}")
print(f"データ数: {len(product_data)}レコード")

if len(product_data) >= 10:  # 最小データ数チェック
    # 特徴量とターゲットを分離
    feature_columns = product_data.select_dtypes(include=['number']).columns.tolist()
    target_column = '数量'
    
    if target_column in feature_columns:
        feature_columns.remove(target_column)
    
    X = product_data[feature_columns]
    y = product_data[target_column]
    
    print(f"特徴量数: {len(feature_columns)}")
    print(f"ターゲット: {target_column}")
    
    # モデル構築
    print("\n🤖 モデル構築中...")
    model_results = model_builder.train_with_cv(X, y)
    
    # 結果表示
    test_metrics = model_results['test_metrics']
    print(f"\n📊 モデル性能:")
    print(f"R²スコア: {test_metrics['r2_score']:.4f}")
    print(f"RMSE: {test_metrics['rmse']:.2f}")
    print(f"MAE: {test_metrics['mae']:.2f}")
    
    # 交差検証結果
    cv_scores = model_results['cv_scores']
    print(f"\n🔄 交差検証:")
    print(f"平均R²: {cv_scores['mean_score']:.4f} ± {cv_scores['std_score']:.4f}")
    
    # 特徴量重要度（上位5位）
    feature_importance = model_results['feature_importance']
    print(f"\n🏆 特徴量重要度 TOP5:")
    for i, (feature, importance) in enumerate(list(feature_importance.items())[:5], 1):
        print(f"{i}. {feature}: {importance:.4f}")
        
else:
    print("⚠️ データ不足のためモデル構築をスキップします")

## 4. カスタマイズ例

### 4.1 カスタム設定での実行

In [ ]:
# カスタム設定の例
custom_config = {
    'model': {
        'n_estimators': 50,  # 決定木数を減らして高速化
        'max_depth': 5,      # 深度を制限して過学習防止
        'cv_folds': 3        # 交差検証フォールド数を削減
    },
    'quality': {
        'thresholds': {
            'premium': 0.6,  # Premium閾値を下げる
            'standard': 0.4,
            'basic': 0.2
        }
    }
}

# 設定を一時ファイルに保存
import yaml
custom_config_path = "../config/custom_config.yaml"
with open(custom_config_path, 'w', encoding='utf-8') as f:
    yaml.dump(custom_config, f, default_flow_style=False, allow_unicode=True)

print("⚙️ カスタム設定でパイプラインを実行")

# カスタム設定でパイプライン実行
custom_pipeline = DemandForecastingPipeline(custom_config_path)
custom_results = custom_pipeline.run_full_analysis(max_products=3)

print(f"カスタム実行結果: {custom_results['summary']['total_products_analyzed']}商品分析")

### 4.2 特定商品のみの分析

In [ ]:
# 特定商品のみ分析
target_products = ['りんご', 'キャベツ']  # 分析したい商品を指定

print(f"🎯 指定商品のみ分析: {', '.join(target_products)}")

# 指定商品が実際にデータに存在するかチェック
available_products = final_features['商品名称'].unique()
valid_products = [p for p in target_products if p in available_products]

if valid_products:
    specific_results = pipeline.run_full_analysis(
        target_products=valid_products,
        max_products=len(valid_products)
    )
    
    print(f"✅ {len(specific_results['analysis_results'])}商品の分析完了")
    
    for result in specific_results['analysis_results']:
        print(f"- {result['product_name']}: {result['quality_level']}")
else:
    print(f"⚠️ 指定された商品がデータに見つかりません")
    print(f"利用可能な商品: {', '.join(available_products[:10])}...")

## 5. 結果の分析と解釈

### 5.1 品質分布の可視化

In [ ]:
# 分析結果から品質分布を抽出
if results['analysis_results']:
    quality_levels = [r['quality_level'] for r in results['analysis_results']]
    r2_scores = [r['test_metrics']['r2_score'] for r in results['analysis_results']]
    product_names = [r['product_name'] for r in results['analysis_results']]
    
    # 品質分布のプロット
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 品質レベル分布
    quality_counts = pd.Series(quality_levels).value_counts()
    colors = {'Premium': '#27AE60', 'Standard': '#F39C12', 'Basic': '#E74C3C', 'Rejected': '#95A5A6'}
    quality_colors = [colors.get(level, '#BDC3C7') for level in quality_counts.index]
    
    ax1.pie(quality_counts.values, labels=quality_counts.index, autopct='%1.1f%%', 
            colors=quality_colors, startangle=90)
    ax1.set_title('品質レベル分布', fontsize=14, fontweight='bold')
    
    # R²スコア分布
    ax2.hist(r2_scores, bins=10, alpha=0.7, color='#FF6B35', edgecolor='black')
    ax2.axvline(np.mean(r2_scores), color='red', linestyle='--', 
                label=f'平均: {np.mean(r2_scores):.3f}')
    ax2.set_xlabel('R²スコア')
    ax2.set_ylabel('商品数')
    ax2.set_title('R²スコア分布', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 統計情報
    print(f"\n📊 分析結果統計:")
    print(f"総商品数: {len(quality_levels)}")
    print(f"平均R²スコア: {np.mean(r2_scores):.4f}")
    print(f"R²スコア標準偏差: {np.std(r2_scores):.4f}")
    print(f"最高R²スコア: {max(r2_scores):.4f} ({product_names[r2_scores.index(max(r2_scores))]})")
    print(f"最低R²スコア: {min(r2_scores):.4f} ({product_names[r2_scores.index(min(r2_scores))]})")
else:
    print("📊 分析結果がないため可視化をスキップします")

### 5.2 レポートファイルの確認

In [ ]:
# 生成されたレポートファイルの確認
print("📄 生成されたファイル一覧:")

print("\n📊 レポートファイル:")
for report_file in results.get('report_files', []):
    file_path = Path(report_file)
    if file_path.exists():
        size = file_path.stat().st_size
        print(f"  ✅ {file_path.name} ({size:,} bytes)")
    else:
        print(f"  ❌ {file_path.name} (ファイルが見つかりません)")

print("\n🎨 可視化ファイル:")
for viz_file in results.get('visualization_files', []):
    file_path = Path(viz_file)
    if file_path.exists():
        size = file_path.stat().st_size
        print(f"  ✅ {file_path.name} ({size:,} bytes)")
    else:
        print(f"  ❌ {file_path.name} (ファイルが見つかりません)")

# Markdownレポートの先頭部分を表示
md_files = [f for f in results.get('report_files', []) if f.endswith('.md')]
if md_files:
    print(f"\n📖 Markdownレポート抜粋 ({Path(md_files[0]).name}):")
    print("-" * 50)
    try:
        with open(md_files[0], 'r', encoding='utf-8') as f:
            content = f.read()
            # 最初の500文字のみ表示
            print(content[:500] + "..." if len(content) > 500 else content)
    except Exception as e:
        print(f"レポート読み込みエラー: {e}")
    print("-" * 50)

## 6. トラブルシューティング

### 6.1 よくある問題と解決方法

In [ ]:
# システム診断機能
def system_diagnosis():
    print("🔍 システム診断を実行中...\n")
    
    # 1. データファイルの確認
    data_path = "../data/raw/納品用_20240101_20241231_生鮮全品data.csv"
    if Path(data_path).exists():
        print("✅ データファイル: 存在します")
        file_size = Path(data_path).stat().st_size
        print(f"   ファイルサイズ: {file_size:,} bytes")
    else:
        print("❌ データファイル: 見つかりません")
        print(f"   期待パス: {data_path}")
    
    # 2. 設定ファイルの確認
    config_path = "../config/config.yaml"
    if Path(config_path).exists():
        print("✅ 設定ファイル: 存在します")
    else:
        print("❌ 設定ファイル: 見つかりません")
        example_path = "../config/config.yaml.example"
        if Path(example_path).exists():
            print(f"   💡 ヒント: {example_path} をコピーして作成してください")
    
    # 3. 出力ディレクトリの確認
    output_dirs = ['../reports', '../output/visualizations', '../logs']
    for dir_path in output_dirs:
        if Path(dir_path).exists():
            print(f"✅ ディレクトリ {Path(dir_path).name}: 存在します")
        else:
            print(f"⚠️ ディレクトリ {Path(dir_path).name}: 存在しません（自動作成されます）")
    
    # 4. Python環境の確認
    print(f"\n🐍 Python環境:")
    print(f"   バージョン: {sys.version.split()[0]}")
    
    # 主要ライブラリの確認
    required_packages = ['pandas', 'numpy', 'scikit-learn', 'matplotlib', 'seaborn']
    for package in required_packages:
        try:
            exec(f"import {package}")
            print(f"   ✅ {package}: インストール済み")
        except ImportError:
            print(f"   ❌ {package}: インストールが必要")
    
    print("\n🔍 診断完了")

# 診断実行
system_diagnosis()

### 6.2 エラー処理のテスト

In [ ]:
# エラー処理のテスト
print("🧪 エラーハンドリングのテスト\n")

# 1. 存在しない設定ファイル
try:
    pipeline_error = DemandForecastingPipeline("nonexistent_config.yaml")
    print("❌ 存在しない設定ファイルのエラーが検出されませんでした")
except Exception as e:
    print(f"✅ 存在しない設定ファイル: 適切にエラーハンドリングされました")
    print(f"   エラー: {type(e).__name__}")

# 2. 不正なデータでの処理
try:
    # 空のデータフレームでの処理テスト
    empty_df = pd.DataFrame()
    processor = DataProcessor(Config())
    result = processor.clean_data(empty_df)
    print(f"✅ 空データの処理: 正常に処理されました（結果: {len(result)}行）")
except Exception as e:
    print(f"⚠️ 空データ処理でエラー: {type(e).__name__}: {e}")

# 3. メモリ使用量の確認
import psutil
import os

process = psutil.Process(os.getpid())
memory_info = process.memory_info()
print(f"\n💾 現在のメモリ使用量: {memory_info.rss / 1024 / 1024:.1f} MB")

print("\n🧪 エラーハンドリングテスト完了")

## まとめ

このノートブックでは、生鮮食品需要予測・分析システムの基本的な使用方法を説明しました。

### 🎯 主要な学習ポイント
1. **パイプライン実行** - `DemandForecastingPipeline`での一括分析
2. **個別コンポーネント** - データ処理、特徴量生成、モデル構築の個別使用
3. **カスタマイズ** - 設定ファイルによる柔軟な調整
4. **結果分析** - 品質評価と可視化による結果の解釈
5. **トラブルシューティング** - 問題診断と解決方法

### 🚀 次のステップ
- 実際のデータでの本格運用
- カスタム特徴量の追加
- モデルパラメータのチューニング
- 定期実行とモニタリングの設定

### 📞 サポート
技術的な質問やトラブルは、プロジェクトの Issues からお気軽にお問い合わせください。